<a href="https://colab.research.google.com/github/jiyoung37/customer_segmentation/blob/main/Customer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing necessary libraries

In [2]:
import numpy as np                                # For numerical operations and handling arrays
import pandas as pd                               # For data manipulation and analysis
import seaborn as sns                             # For statistical data visualization (e.g., heatmaps)
import matplotlib.pyplot as plt                   # For visualizing data
from sklearn.preprocessing import MinMaxScaler    # For scaling the data (Min-Max)
from sklearn.preprocessing import StandardScaler  # For scaling the data (Standardization)
from sklearn.cluster import KMeans                # For K-Means clustering
from sklearn.cluster import DBSCAN                # For DBSCAN clustering
from sklearn.metrics import silhouette_score      # For evaluating clustering performance
from sklearn.manifold import TSNE                 # For dimensionality reduction (t-SNE)
from sklearn.decomposition import PCA
import os                                         # For Traversing through directories to find and access files (useful for Kaggle Kernels)
import warnings


# Ignore all warnings
warnings.filterwarnings('ignore')

# Data preparation

Download file from Github

In [8]:
# Load dataset from Github
import pandas as pd

# Use the correct raw GitHub link
url = "https://raw.githubusercontent.com/jiyoung37/customer_segmentation/main/Online%20Retail.xlsx"

# Read Excel file with the correct engine
df = pd.read_excel(url, engine="openpyxl")

# Display first few rows
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [15]:
import requests

# Correct raw GitHub link
url = "https://raw.githubusercontent.com/jiyoung37/customer_segmentation/main/Online%20Retail.xlsx"
file_name = "Retail_c.xlsx"

# Download the file
response = requests.get(url)
with open(file_name, "wb") as file:
    file.write(response.content)

# Read the downloaded Excel file
df = pd.read_excel(file_name, engine="openpyxl")

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


# Data overview

In [21]:
# data shape
print("data shape:",df.shape)

data shape: (541909, 8)


In [17]:
# missing data
df.isna().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [34]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID,decimal_part
count,541909.000000,541909,541909.000000,541909.000000,406829.0
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,11476.974671,0.0
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,0.000000,0.0
25%,1.000000,2011-03-28 11:34:00,1.250000,12352.000000,0.0
50%,3.000000,2011-07-19 17:17:00,2.080000,14382.000000,0.0
75%,10.000000,2011-10-19 11:27:00,4.130000,16255.000000,0.0
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000,0.0
std,218.081158,NaN,96.759853,6777.908326,0.0


# Data cleaning

Change type: CustomerID - float to int

In [30]:
non_integer_count = (df['CustomerID'] % 1 != 0).sum()
print(f"Number of non-integer customer IDs: {non_integer_count}")

df['decimal_part'] = df['CustomerID'] % 1
decimal_values = df['decimal_part'].unique()
print("Unique decimal values:", decimal_values)

print("The unique decimal values are only 0 and nan. Therefore we can convert the CustomerID to integer type")

Number of non-integer customer IDs: 135080
Unique decimal values: [ 0. nan]


In [33]:
df.CustomerID = df.CustomerID.fillna(0).astype(int)
print(f"The type of CustomerID is {df.CustomerID.dtype}.")

The type of CustomerID is int64.


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     541909 non-null  object        
 1   StockCode     541909 non-null  object        
 2   Description   540455 non-null  object        
 3   Quantity      541909 non-null  int64         
 4   InvoiceDate   541909 non-null  datetime64[ns]
 5   UnitPrice     541909 non-null  float64       
 6   CustomerID    541909 non-null  int64         
 7   Country       541909 non-null  object        
 8   decimal_part  406829 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 37.2+ MB
